In [ ]:
pip install yargy

In [3]:
from yargy import Parser, rule, and_, not_, or_
from yargy.interpretation import fact
from yargy.relations import gnc_relation
from yargy.predicates import gram, is_capitalized, type, caseless, eq, custom, normalized
from yargy.tokenizer import Tokenizer

In [4]:
import unittest
import pandas as pd

### Yargy

In [5]:
gnc = gnc_relation()  # согласование по gender, number и case (падежу, числу и роду)

In [6]:
# Объект-результат интерпретации
PersonInformation = fact(
    'PersonInformation',
    ['first', 'last', 'patronymic']
)

В словаре Opencorpora, который использует pymorphy2, для имён ставится метка `Name`, для фамилий — метка `Surn`, для отчеств - метка `Patr`.

In [7]:
FIRST = gram('Name').interpretation(PersonInformation.first).match(gnc)

In [8]:
LAST = gram('Surn').interpretation(PersonInformation.last).match(gnc)

In [9]:
PATRONYMIC = gram('Patr').interpretation(PersonInformation.patronymic).match(gnc)

In [10]:
# случай, когда после "по" идет существительное в дательном падеже -> это предлог, а не фамилия (Есть фамилия писателя Эдгар Аллан По)
PREPOSITION = rule(
          caseless('по'),
          not_(eq(gram('NOUN').interpretation(PersonInformation.first.inflected('datv'))))
    )

In [11]:
def is_surname(name):
  if ((name.endswith('ов') == True or name.endswith('ова') == True) or
     (name.endswith('ев') == True or name.endswith('ева') == True) or
     (name.endswith('ин') == True or name.endswith('ина') == True) or
     (name.endswith('ын') == True or name.endswith('ына') == True) or
     (name.endswith('их') == True) or (name.endswith('ых') == True) or
     (name.endswith('ский') == True or name.endswith('ская') == True) or
     (name.endswith('цкий') == True or name.endswith('цкая') == True)):
    return name

In [12]:
# фамилия, которую Opencorpora распознает как имя (но содержит суффикс фамилии)
# ex. 'хамидов'
SURNAME_LIKE_NAME = gram('Name').interpretation(PersonInformation.last.custom(is_surname))

In [13]:
# фамилия, которую Opencorpora распознает как существительное (но содержит суффикс фамилии)
# ex. 'терминов'
SURNAME_LIKE_NOUN = gram('NOUN').interpretation(PersonInformation.last.custom(is_surname))

In [14]:
# фамилия, которую Opencorpora распознает как существительное, но без суффикса фамилии
# ex. 'школьник нина викторовна'
SURNAME_LIKE_NOUN_WITHOUT = rule(
    and_(  
        gram('NOUN'), gram('sing'), gram('nomn')
    ).interpretation(PersonInformation.last)
)

In [15]:
# фамилия, которую Opencorpora распознает как наречие, но без суффикса фамилии (у наречия больше шансов быть фамилией, если оно произошло от краткого прилагательного)
# ex. 'грешно максим владимирович'
SURNAME_LIKE_ADVB_WITHOUT = rule(
    and_(
        gram('ADVB'), gram('ADJS')
    ).interpretation(PersonInformation.last)
)

In [16]:
PERSON_INFO = or_(
    # whole
    rule(FIRST, LAST, PATRONYMIC),
    rule(LAST, FIRST, PATRONYMIC),
    rule(FIRST, PATRONYMIC, LAST),
    rule(SURNAME_LIKE_NOUN_WITHOUT, FIRST, PATRONYMIC),
    rule(SURNAME_LIKE_NAME, FIRST, PATRONYMIC),
    rule(SURNAME_LIKE_NOUN, FIRST, PATRONYMIC),  
    rule(SURNAME_LIKE_ADVB_WITHOUT, FIRST, PATRONYMIC),
    # two words
    rule(FIRST, LAST),
    rule(LAST, FIRST),
    rule(FIRST, PATRONYMIC),
    rule(SURNAME_LIKE_NAME, FIRST),
    rule(FIRST, SURNAME_LIKE_NAME),
    # one word
    PREPOSITION,
    rule(FIRST),
    rule(LAST),
    rule(PATRONYMIC)
).interpretation(PersonInformation)

In [17]:
parser = Parser(PERSON_INFO)

In [18]:
class TestPersons(unittest.TestCase):
    def setUp(self):
        self.Parser = Parser(PERSON_INFO)

    def test_1(self):
        testing_address = 'Иванов Петр Васильевич'
        res = parser.find(testing_address).fact
        self.assertEqual(res.first, 'Петр')
        self.assertEqual(res.patronymic, 'Васильевич')
        self.assertEqual(res.last, 'Иванов')

    def test_2(self):
        testing_address = 'шипицын дмитрий вячеславович'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, 'дмитрий')
        self.assertEqual(res.patronymic, 'вячеславович')
        self.assertEqual(res.last, 'шипицын')

    def test_3(self):
        testing_address = 'елена владимировна'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, 'елена')
        self.assertEqual(res.patronymic, 'владимировна')
        self.assertEqual(res.last, None)

    def test_4(self):
        testing_address = 'басалаева юлия михайловна'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, 'юлия')
        self.assertEqual(res.patronymic, 'михайловна')
        self.assertEqual(res.last, 'басалаева')

    def test_5(self):
        testing_address = 'ну я как раз по фамилии есть смотри мам'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, None)
        self.assertEqual(res.patronymic, None)
        self.assertEqual(res.last, None)

    def test_6(self):
        testing_address = 'глушенков власти на android'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, None)
        self.assertEqual(res.patronymic, None)
        self.assertEqual(res.last, 'глушенков')

    def test_7(self):
        testing_address = 'фамилию сказать что за фамилия терентьева людмила'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, 'людмила')
        self.assertEqual(res.patronymic, None)
        self.assertEqual(res.last, 'терентьева')

    def test_8(self):
        testing_address = 'елена владимировна'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, 'елена')
        self.assertEqual(res.patronymic, 'владимировна')
        self.assertEqual(res.last, None)

    def test_9(self):
        testing_address = 'анюта'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, 'анюта')
        self.assertEqual(res.patronymic, None)
        self.assertEqual(res.last, None)

    def test_10(self):
        testing_address = 'р1 артем витальевич'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, 'артем')
        self.assertEqual(res.patronymic, 'витальевич')
        self.assertEqual(res.last, None)
        
    def test_11(self):
        testing_address = 'фитнес веретельников олег викторович'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.first, 'олег')
        self.assertEqual(res.patronymic, 'викторович')
        self.assertEqual(res.last, 'веретельников')

In [19]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

...........
----------------------------------------------------------------------
Ran 11 tests in 1.637s

OK
